#####  环境： ubuntu  python27 tensorflowf

In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'1.13.1'

### 3.1.2 计算图的使用

#### Tensorflow程序执行分两步：定义计算图所有的计算，以及执行计算，如下cell是第一步

In [3]:
a = tf.constant([1.0,2.0],name="a")
b = tf.constant([2.0,3.0],name="b")
result = a+b

#### 获取默认计算图，并查看一个运算所属的计算图

In [4]:
print(a.graph is tf.get_default_graph())

True


In [5]:
print(tf.get_default_graph())

#### 还支持tf.Graph函数生成新的计算图，不同计算图上的张量和运算不共享

In [8]:
import tensorflow as tf

g1 = tf.Graph()
with g1.as_default():
    v = tf.get_variable("v",shape=[1,6],initializer=tf.zeros_initializer)
    
g2 = tf.Graph()
with g2.as_default():
    v = tf.get_variable("v",shape=[2,5],initializer=tf.ones_initializer)

with tf.Session(graph=g1) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("",reuse=True):
        print(sess.run(tf.get_variable("v")))

with tf.Session(graph=g2) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("",reuse=True):
        print(sess.run(tf.get_variable("v")))

[[0. 0. 0. 0. 0. 0.]]
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]


## 3.2 Tensorflow 数据模型-张量

###  3.2.1 张量的概念

In [14]:
import tensorflow as tf
a = tf.constant([1.0,2.0],name="a")
b = tf.constant([2.0,3.0],name="b")
result = tf.add(a,b,name="add")
print(result)

Tensor("add_4:0", shape=(2,), dtype=float32)


In [17]:
a = tf.constant([11.0,21.0],name="a")
b = tf.constant([21.0,31.0],name="b")
result = tf.add(a,b,name="add")

print(result)

Tensor("add_6:0", shape=(2,), dtype=float32)


In [18]:
a = tf.constant([1.0,2.0],name="a")
b = tf.constant([2,3],name="b")
result = tf.add(a,b,name="add")
print(result)

TypeError: Input 'y' of 'Add' Op has type int32 that does not match type float32 of argument 'x'.

## 3.3 TensorFlow运行模型-会话

In [13]:
g3 = tf.Graph()
with g3.as_default():
    a = tf.constant([1.0,2.0],name="a")
    b = tf.constant([2.0,3.0],name="b")
    result = a+b
    
sess = tf.Session(graph=g3)
result_out = sess.run(result)
sess.close()

print(result_out)

[3. 5.]


#### 如上cell中，需要明确调用Session.close()函数关闭并释放资源。但是程序因异常退出的时候，关闭函数就可能不会被执行而导致资源泄露。
#### 为解决该问题，采用python上下文管理器来管理会话。

In [20]:
a = tf.constant([1.0,22.0],name="a")
b = tf.constant([4.0,3.0],name="b")
result = a+b

with tf.Session() as sess:
    sess.run(result)

####  计算张量的取值

In [21]:
sess = tf.Session()
with sess.as_default():
    print(result.eval())

[ 5. 25.]


In [20]:
#  上面cell代码的另一种实现
sess = tf.Session()

print(sess.run(result))
print(result.eval(session=sess))
sess.close()

[32. 52.]
[32. 52.]


#### 在交互式环境下可以直接构建默认会话

In [23]:
sess = tf.InteractiveSession()
print(result.eval())
sess.close()

[ 5. 25.]


## 3.4 TensorFlow实现神经网络

P53两层神经网络的前向传播过程

In [ ]:
g3 = tf.Graph()
with g3.as_default():
    a = tf.constant([1.0,2.0],name="a")
    b = tf.constant([2.0,3.0],name="b")
    result = a+b
    
sess = tf.Session(graph=g3)
result_out = sess.run(result)
sess.close()

print(result_out)

In [39]:
# 一个变量在被使用之前，这个变量的初始化过程需要被明确地调用
g053119171018 = tf.Graph()
with g053119171018.as_default():
    weights = tf.Variable(tf.random_normal([2,3],stddev=2))
    w2 = tf.Variable(weights.initialized_value())
    w3 = tf.Variable(weights.initialized_value()*2.0)
    x = tf.fill([2,3],9)

In [40]:
with tf.Session(graph=g053119171018) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    print(w2.eval())
    print(w3.eval())
    print(weights.eval())
    print(x.eval())

[[-1.2867814  -1.399865    0.34962618]
 [-0.31424052  2.0316892  -0.27023044]]
[[-2.5735629  -2.79973     0.69925237]
 [-0.62848103  4.0633783  -0.5404609 ]]
[[-1.2867814  -1.399865    0.34962618]
 [-0.31424052  2.0316892  -0.27023044]]
[[9 9 9]
 [9 9 9]]


###  神经网络前向传播

In [ ]:
# 版本一：
import tensorflow as tf
w1 = tf.Variable(tf.random_normal((2,3),stddev=1,seed=1))
w2 = tf.Variable(tf.random_normal((3,1),stddev=1,seed=1))

x = tf.constant([[0.7,0.9]])

a = tf.matmul(x,w1)
y = tf.matmul(a,w2)

sess = tf.Session()

# error：sess.run(w1.initialized_value()) 
sess.run(w1.initializer)
sess.run(w2.initializer)
'''
# 如上对w1和w2单独进行初始化，现在可通过下面语句直接对着两个变量初始化
init_op = tf.global_variables_initializer()
sess.run(init_op)
'''
print(w1.eval(session=sess))
print('y=%f'%(sess.run(y)))
sess.close()

In [51]:
# 版本2
import tensorflow as tf
w1 = tf.Variable(tf.random_normal((2,3),stddev=1,seed=1))
w2 = tf.Variable(tf.random_normal((3,1),stddev=1,seed=1))

x = tf.constant([[0.7,0.9]])

a = tf.matmul(x,w1)
y = tf.matmul(a,w2)

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    y_value0 = sess.run(y)

In [52]:
print(y_value0)

[[3.957578]]


#### 一个变量在构建之后，它的类型就不能再改变了

In [53]:
w1 = tf.Variable(tf.random_normal([2,3],stddev=1,seed=1),name="w1")
w2 = tf.Variable(tf.random_normal((2,3),dtype = tf.float64,stddev=1,seed=1),name="w2")

In [54]:
print(w1,w2)

(<tf.Variable 'w1:0' shape=(2, 3) dtype=float32_ref>, <tf.Variable 'w2:0' shape=(2, 3) dtype=float64_ref>)


In [ ]:
#w1.assign(w2) # 会报错

#### 维度在程序运行中是有可能改变的，但是要设置validate_shape = False

In [72]:
sess = tf.Session()
w1 = tf.Variable(tf.zeros([3,3]),name="w1")
w2 = tf.Variable(tf.ones([2,2]),name="w2")

In [73]:
# tf.assign(w1,w2) #会报错

In [74]:
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print( sess.run(w1) )
    print( sess.run(w2) )

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[1. 1.]
 [1. 1.]]


In [75]:
 tf.assign(w1,w2,validate_shape = False) 

<tf.Tensor 'Assign_3:0' shape=(2, 2) dtype=float32_ref>

In [76]:
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print( sess.run(w1) )
    print( sess.run(w2) )

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[1. 1.]
 [1. 1.]]


### 3.4.4 通过TensorFlow训练神经网络模型

In [77]:
import tensorflow as tf
w1 = tf.Variable(tf.random_normal((2,3),stddev=1,seed=1))
w2 = tf.Variable(tf.random_normal((3,1),stddev=1,seed=1))

x = tf.placeholder(tf.float32,shape=(1,2),name="input")
a = tf.matmul(x,w1)
y = tf.matmul(a,w2)

sess = tf.Session()

init_op = tf.global_variables_initializer()
sess.run(init_op)

print(sess.run(y,feed_dict={x:[[0.7,0.9]]}))
sess.close()

[[3.957578]]


In [ ]:
y = tf.sigmoid(y)
cross_entropy = -tf.reduce_mean(
    y_*tf.log(tf.clip_by_value(y,1e-10,1.0))
    +(1-y_)*tf.log(tf.clip_by_value(1-y,1e-10,1.0))
)

learning_rate = 0.001
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

##### 注：tf.reduce_mean的用法

In [85]:
a=tf.constant([[1.0,2.0],[3.0,4.0]])
b = tf.reduce_mean(a)#tf.reduce_mean(a,axis=1)
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    print(sess.run(b))

2.5


### 3.4.5 完整神经网络样例程序

In [ ]:
import tensorflow as tf
from numpy.random import RandomState
import numpy as np
batch_size = 64

w1Size = 20# 取数大一点，为了验证一下tf可以用多核
w2Size = 30
w1 = tf.Variable(tf.random_normal((w1Size,w2Size),stddev=1,seed=1))
w2 = tf.Variable(tf.random_normal((w2Size,1),stddev=1,seed=1))

#在shape维度使用none方便使用不同batch的大小
x = tf.placeholder(tf.float32,shape=(None,w1Size),name="x-input")
y_ = tf.placeholder(tf.float32,shape=(None,1),name="y-input")

# 前向
a = tf.matmul(x,w1)
y = tf.matmul(a,w2)

# 定义损失函数和反向传播
y = tf.sigmoid(y)
cross_entropy = -tf.reduce_mean(
    y_*tf.log(tf.clip_by_value(y,1e-10,1.0))
    +(1-y_)*tf.log(tf.clip_by_value(1-y,1e-10,1.0))
)

learning_rate = 0.001
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

#通过随机数生成数据集
rdm = RandomState(1)
dataset_size = 12800
X = rdm.rand(dataset_size,w1Size)
#定义规则样本标签。所有x1+x2<1认为是正样本表示为1，负样本表示为0
# Y = [[int(x1+x2<1) ]for (x1,x2) in X]
Y = np.zeros((dataset_size,1))
for i,ele in enumerate(X):
    if np.sum(ele)<100:
        Y[i] = 1
    else:
        Y[i] = 0
print ("rand dataset has constructed")
        
with tf.Session(config=tf.ConfigProto(
#device_count={"CPU":63},#感觉指定与否区别不大
inter_op_parallelism_threads=1,
intra_op_parallelism_threads=1,
)) as sess:
    init_op = tf.global_variables_initializer()
    #初始化变量
    sess.run(init_op)
    
    #print sess.run(w1)
    #print sess.run(w2)
    
    STEPS = 50000
    for i in range(STEPS):
        #每次选取batch个样本进行训练
        start = (i*batch_size)%dataset_size
        end = min(start+batch_size,dataset_size)
        
        #通过选取的样本外训练神经网络并更新参数
        sess.run(train_step,feed_dict={x:X[start:end],y_:Y[start:end]})
        if i%5000 == 0:
            total_cross_entropy = sess.run(cross_entropy,feed_dict={x:X,y_:Y})
            print("After %d training step(s),cross entropy on all data is %g"%\
                 (i,total_cross_entropy))